<a href="https://colab.research.google.com/github/288756/VisArtificial/blob/master/Miniproyecto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from google.colab import drive
from sklearn.preprocessing import LabelEncoder
import os
from PIL import Image
import tensorflow as tf
from tensorflow.data import Dataset
import matplotlib.pyplot as plt
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input
import numpy as np
import tensorflow_addons as tfa
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Montar el Google Drive en el directorio del proyecto y descomprimir el fichero con los datos
drive.mount('/content/gdrive')
!unzip -n '/content/gdrive/My Drive/vision-artificial.zip' >> /dev/null  # ACTUALIZAR: ruta al fichero comprimido


/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


Mounted at /content/gdrive


In [25]:
!pip install tensorflow_addons

In [26]:
# Especificar las rutas al directorio con las imágenes y al fichero con las etiquetas
imgtrain_dir = "/content/train/images/"
imgtrainmask_dir = "/content/train/masks/"
csvtrain_file = "/content/train.csv"

imgtest_dir = "/content/train.csv"
csvtest_file = "/content/test.csv"
# Leer el fichero CSV con las etiquetas
df = pd.read_csv(csvtrain_file, dtype={"class": "category"})
dftest = pd.read_csv(csvtest_file, dtype={"class": "category"})

# Codificar las etiquetas utilizando LabelEncoder
label_encoder = LabelEncoder()
df['class_encoded'] = label_encoder.fit_transform(df['class'])

# Convertir las etiquetas codificadas en un vector one-hot
class_one_hot = pd.get_dummies(df['class_encoded'], prefix='class')

# Renombrar las columnas del vector one-hot
class_one_hot.columns = ['normal', 'benign', 'malignant']

# Concatenar el DataFrame original con las etiquetas one-hot y la columna 'mask_filename'
df = pd.concat([df[['image_filename', 'mask_filename']], class_one_hot], axis=1)

In [ ]:
column_names = df.columns.tolist()
print(df)
print(column_names[2:5])

    image_filename mask_filename  normal  benign  malignant
0    image_003.png  mask_003.png    True   False      False
1    image_004.png  mask_004.png    True   False      False
2    image_008.png  mask_008.png    True   False      False
3    image_012.png  mask_012.png    True   False      False
4    image_016.png  mask_016.png    True   False      False
..             ...           ...     ...     ...        ...
332  image_399.png  mask_399.png   False   False       True
333  image_400.png  mask_400.png   False   False       True
334  image_416.png  mask_416.png   False   False       True
335  image_419.png  mask_419.png   False   False       True
336  image_443.png  mask_443.png   False   False       True

[337 rows x 5 columns]
['normal', 'benign', 'malignant']


In [27]:
# Dividir el conjunto en entrenamiento, validación y test (80:10:10)
val_size = int(len(df) * 0.15)
test_size = int(len(df) * 0.15)

df = df.sample(frac=1).reset_index(drop=True)  # barajar el dataframe
dftest = df[:test_size]
dfval = df[test_size:test_size+val_size]
dftrain = df[test_size+val_size:]

print(f'Número de ejemplos del conjunto de entrenamiento: {dftrain.shape[0]}')
print(f'Número de ejemplos del conjunto de validación: {dfval.shape[0]}')
print(f'Número de ejemplos del conjunto de test: {dftest.shape[0]}')
dftrain = dftrain.reset_index(drop=True)
dfval = dfval.reset_index(drop=True)
dftest = dftest.reset_index(drop=True)

Número de ejemplos del conjunto de entrenamiento: 237
Número de ejemplos del conjunto de validación: 50
Número de ejemplos del conjunto de test: 50


In [ ]:
dftrain

,image_filename,mask_filename,normal,benign,malignant
0,image_418.png,mask_418.png,True,False,False
1,image_075.png,mask_075.png,True,False,False
2,image_065.png,mask_065.png,False,True,False
3,image_327.png,mask_327.png,False,True,False
4,image_046.png,mask_046.png,True,False,False
...,...,...,...,...,...
232,image_273.png,mask_273.png,False,False,True
233,image_256.png,mask_256.png,False,True,False
234,image_161.png,mask_161.png,False,False,True
235,image_112.png,mask_112.png,False,True,False


In [ ]:
archivos = os.listdir(imgtrain_dir)
anchuras = []
alturas = []
for archivo in archivos:
      imagen = Image.open(os.path.join(imgtrain_dir, archivo))
      ancho, alto = imagen.size
      anchuras.append(ancho)
      alturas.append(alto)
media_anchura = sum(anchuras) / len(anchuras)
media_altura = sum(alturas) / len(alturas)

print("Media de anchura:", media_anchura)
print("Media de altura:", media_altura)

Media de anchura: 634.9139465875371
Media de altura: 514.9317507418398


In [28]:
# Dimensiones deseadas de la imagen
img_width, img_height = 244,244
n_channels = 3                # número de canales (RGB)
n_classes = 3                 # número de clases
x_col = 'image_filename'      # nombres de las columnas en el fichero CSV
y_col = ['normal', 'benign', 'malignant']  # lista de nombres de las columnas de las etiquetas

def load_and_preprocess_image(image_filename, label):
    image_path = tf.strings.join([imgtrain_dir, image_filename])
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=n_channels)

    # Aumento de datos: rotación aleatoria
    # image = tfa.image.rotate(image, tf.random.uniform(shape=[], minval=-5, maxval=5))

    image = tf.image.resize(image, [img_width, img_height])
    image = image / 255.0
    return image, label

# Crear conjunto de datos
def get_dataset(df):
    image_paths = df[x_col].values
    labels = df[y_col].values
    dataset = Dataset.from_tensor_slices((image_paths, labels))
    dataset = dataset.map(load_and_preprocess_image)
    return dataset

# Crear los conjuntos de datos y preparar los lotes
batch_size = 32
train_dataset = get_dataset(dftrain).batch(batch_size)
val_dataset = get_dataset(dfval).batch(batch_size)
test_dataset = get_dataset(dftest).batch(batch_size)

print(f'Número de lotes del conjunto de entrenamiento: {len(train_dataset)}')
print(f'Número de lotes del conjunto de validación: {len(val_dataset)}')
print(f'Número de lotes del conjunto de test: {len(test_dataset)}')





Número de lotes del conjunto de entrenamiento: 8
Número de lotes del conjunto de validación: 2
Número de lotes del conjunto de test: 2


In [32]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras import backend as K

def get_model():

  # Cargar la base convolucional del modelo VGG16 pre-entrenado en ImageNet
  base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_width,img_height,3))

  # Ajustar los parámetros de las nuevas capas del modelo, dejando fijos los parámetros del resto de capas
  for layer in base_model.layers:
      layer.trainable = False   # por defecto, layer.trainable es True

  # Añadir nuevas capas a continuación de la base convolucional, para resolver la tarea de aprendizaje
  x = base_model.output
  x = GlobalAveragePooling2D()(x)
  x = Dense(256, activation='relu')(x)

  # Añadir una última capa completamente conectada con 5 neuronas (número de clases) para obtener la salida de la red
  predictions = Dense(n_classes, activation='softmax')(x)

  # Crear el modelo final e imprimir su representacion en modo texto
  model = Model(inputs=[base_model.input], outputs=[predictions])
  model.summary()

  return model

def f1_metric(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    precision = tp / (tp + fp + K.epsilon())
    recall = tp / (tp + fn + K.epsilon())

    f1 = 2 * precision * recall / (precision + recall + K.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)

    # Compute the weighted F1 score
    weights = K.sum(y_true, axis=0) / K.sum(y_true)
    weighted_f1 = K.sum(weights * f1)

    return weighted_f1

# Definir el tamaño de entrada y el número de clases
input_shape = (img_width, img_height, n_channels)
num_classes = 3  # Normal, Benigno, Maligno
model = get_model()

model.compile(loss='categorical_crossentropy',  # función de pérdida para problemas de clasificación multi-clase
              optimizer="adam",
              metrics=[f1_metric])

# Imprimir el resumen del modelo
model.summary()


Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 244, 244, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 244, 244, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 244, 244, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 122, 122, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 122, 122, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 122, 122, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 61, 61, 128)       0   

In [36]:
early_stopping = EarlyStopping(monitor='val_accuracy', patience=20, mode='max')
# reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.2, patience=5, min_lr=0.00001) Pocas épocas para que se note

model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=20,
    verbose = 2,
    steps_per_epoch=np.ceil(len(dftrain) / batch_size),
    validation_steps=np.ceil(len(dfval) / batch_size),
    callbacks=[early_stopping]#, reduce_lr]
)


Epoch 1/20


8/8 - 2s - loss: 0.2676 - f1_metric: 0.9053 - val_loss: 0.8607 - val_f1_metric: 0.6114 - 2s/epoch - 278ms/step
Epoch 2/20


8/8 - 2s - loss: 0.2630 - f1_metric: 0.9075 - val_loss: 0.8623 - val_f1_metric: 0.6114 - 2s/epoch - 228ms/step
Epoch 3/20


8/8 - 2s - loss: 0.2586 - f1_metric: 0.9075 - val_loss: 0.8624 - val_f1_metric: 0.6114 - 2s/epoch - 229ms/step
Epoch 4/20


8/8 - 2s - loss: 0.2539 - f1_metric: 0.9097 - val_loss: 0.8640 - val_f1_metric: 0.6114 - 2s/epoch - 229ms/step
Epoch 5/20


...

8/8 - 2s - loss: 0.2058 - f1_metric: 0.9304 - val_loss: 0.8812 - val_f1_metric: 0.6114 - 2s/epoch - 232ms/step
Epoch 17/20


8/8 - 2s - loss: 0.2020 - f1_metric: 0.9343 - val_loss: 0.8826 - val_f1_metric: 0.5960 - 2s/epoch - 202ms/step
Epoch 18/20


8/8 - 2s - loss: 0.1985 - f1_metric: 0.9343 - val_loss: 0.8853 - val_f1_metric: 0.5960 - 2s/epoch - 234ms/step
Epoch 19/20


8/8 - 2s - loss: 0.1950 - f1_metric: 0.9364 - val_loss: 0.8859 - val_f1_metric: 0.6233 - 2s/epoch - 246ms/step
Epoch 20/20


8/8 - 2s - loss: 0.1915 - f1_metric: 0.9422 - val_loss: 0.8893 - val_f1_metric: 0.6233 - 2s/epoch - 202ms/step


In [37]:
test_loss, test_acc = model.evaluate(test_dataset, verbose=2)
print("test_loss: %.4f, test_acc: %.4f" % (test_loss, test_acc))

2/2 - 0s - loss: 0.7302 - f1_metric: 0.6884 - 455ms/epoch - 227ms/step
test_loss: 0.7302, test_acc: 0.6884


In [38]:
# Directorio donde se encuentran las imágenes de prueba
test_images_dir = 'test/images/'

# Obtener la lista de nombres de archivos de las imágenes de prueba
test_filenames = os.listdir(test_images_dir)

# Crear un DataFrame para almacenar las predicciones
predictions_df = pd.DataFrame({'image_filename': test_filenames})

# Crear columnas para 'normal', 'benign' y 'malignant' con valores iniciales de 0
predictions_df['normal'] = 0
predictions_df['benign'] = 0
predictions_df['malignant'] = 0

# Iterar sobre cada imagen de prueba
for filename in test_filenames:
    # Cargar la imagen y preprocesarla
    img_path = os.path.join(test_images_dir, filename)
    img = load_img(img_path, target_size=(img_width, img_height))
    img_array = img_to_array(img)
    img_array = preprocess_input(img_array)
    img_array = np.expand_dims(img_array, axis=0)  # Añadir una dimensión adicional para el lote

    # Realizar la predicción
    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction)  # Obtener la clase predicha

    # Actualizar las columnas correspondientes según la predicción
    if predicted_class == 0:
        predictions_df.loc[predictions_df['image_filename'] == filename, 'normal'] = 1
    elif predicted_class == 1:
        predictions_df.loc[predictions_df['image_filename'] == filename, 'benign'] = 1
    elif predicted_class == 2:
        predictions_df.loc[predictions_df['image_filename'] == filename, 'malignant'] = 1

# Ordenar el DataFrame por el nombre del archivo de imagen
predictions_df = predictions_df.sort_values(by='image_filename')

# Guardar el DataFrame en un archivo CSV
predictions_df.to_csv('test_predictions.csv', index=False)

# Imprimir las primeras filas del DataFrame para verificar
print(predictions_df.head())


1/1 [==============================] - 0s 20ms/step
   image_filename  normal  benign  malignant
74  image_005.png       1       0          0
32  image_006.png       1       0          0
14  image_007.png       0       1          0
2   image_009.png       1       0          0
92  image_010.png       1       0          0
